In [ ]:
from flask_sqlalchemy import SQLAlchemy
from pandas.io.parsers import read_csv
import plotly.express as px
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import pandas as pd
import numpy as np
import json
import requests
import dash_table
import jupyter_dash as JupyterDash
import random as r


#for Heroku Flask deployment
from flask_sqlalchemy import SQLAlchemy
from flask import Flask
from jupyter_dash import JupyterDash

import dash_bootstrap_components as dbc

import locale
locale.setlocale(locale.LC_ALL, '')  # Use '' for auto, or force e.g. to 'en_US.UTF-8'

# external_stylesheets = [dbc.themes.YETI]

### FULL APP AS OF 7/1/2021

In [ ]:
# #Population Dropdown
# metric_dropdown = dbc.Col(
#     dcc.Dropdown(
#         id = 'population-dropdown',
#         options= [
#             {'label': '< 5 Million' , 'value': 2},
#             {'label': '5-10 Million', 'value': 1},
#             {'label': 'More than 5 Million', 'value': 0}
#         ],
#         value=[],
#         multi=True,
#         optionHeight = 25
#     ),
#     xs = 12,
#     md = 5, 
# )


In [ ]:
#LOAD WHEN WEBSITE LOADS

country_reference = pd.read_csv('/Users/ems/Documents/Coronavirus_Dashboard/geodata/country_reference.csv')
df = pd.read_csv('/Users/ems/Documents/df.csv', parse_dates = ['Date'])
available_countries = country_reference.loc[(country_reference['ISO-3'].isin(df['ISO-3'])), ['Country', 'ISO-3']].drop_duplicates().values

In [ ]:
#parse date (sigh)
# country_data['Date'] =  pd.to_datetime(country_data['Date'].str.slice(0,10)) # + " " + country_data['Date'].str.slice(11, -1)
# country_data['Month and Year'] = pd.DatetimeIndex(country_data['Date']).strftime("%b %Y")



country_card_list = dbc.Col(
    id = 'country-card-list',
    style = {
#         'border': 'none', 
        'fontSize' : '14px',
        'backgroundColor' : '#ede7c6',
        'paddingTop' : '20px' 
    },
    width = 4
)


In [ ]:
#Country Dropdown
country_dropdown_country = dbc.Col( 
    children = [
        html.Label(html.H5('Countries')),
        dcc.Dropdown(
            id = 'country-dropdown-country',
            options = [{'label': country, 'value' : iso} for country, iso in available_countries],
            style = {'fontSize': '14px'}, 
            value = 'USA', 
            optionHeight = 22, 
            clearable = False
        )
    ],
    xs = 12,
    md = 6,
)

In [ ]:
#Choropleth Map for Country Tab
focus_map = dbc.Col(
    dcc.Graph(id = 'focus-map'), 
    width = 8, 
    style={'position':'relative', 'zIndex':'1', 'paddingTop' : '20px'}, 
    
)

region_line = dbc.Col(
    dcc.Graph(id = 'region-line'),
    width = 12
)

In [ ]:
#Tabs
country_card_description = dbc.Col(id = 'country-card-description', width = 12)

country_tab =  dcc.Tab(
    id = 'country-tab',
    value = 'country-tab',
    label='Country Focus',  
    children = dbc.Container(
        [dbc.Row([focus_map, country_card_list]),
        dbc.Row(country_card_description), 
        dbc.Row(region_line)])
)

global_tab = dcc.Tab(id = 'global-tab', 
                     label='Global Overview', 
                     value='global-tab')
        
tabs = dcc.Tabs(
        id = 'tabs', 
        value='country-tab', 
        children=[country_tab, global_tab], 
        colors={ "border": "#2F4F4F", "primary": "ede712", "background": "#ede7c6" }
)

controls_global = html.Div([
        dbc.Container([
            dbc.Row([
    html.H5('GLOBAL!!!')
            ])]
        ),
    ], 
        id  = 'controls-global'
)

controls_country = html.Div([
        dbc.Container([
            dbc.Row([
                country_dropdown_country,
                metric_dropdown_country,
            ]),
            ]
        ),
    ], 
        id = 'controls-country'
)

controls_container = html.Div(
    id = 'controls-container', 
    className = 'controls',
    children = [controls_country, controls_global])


In [ ]:
metric_dropdown_country = dbc.Col(
    children = [
    html.Label(html.H5('Select a Country')),   
    dcc.Dropdown(
        id = 'metric-dropdown-country',
        options = [
            # Confirmed Case Metrics
            {'label':'Cumulative Cases', 'value': 'Cases'},
            {'label':'Cumulative Cases per 1M', 'value': 'Cases per 1M'}, 
            {'label':'Daily Cases (n)', 'value': 'New Cases (n)'},
            {'label':'Daily Cases (Smoothed)', 'value':'New Cases (SMA)'},

            #Confirmed Death Metrics
            {'label':'Cumulative Deaths', 'value': 'Deaths'},
            {'label':'Cumulative Deaths per 1M', 'value': 'Deaths per 1M'},
            {'label':'Daily Deaths (n)', 'value': 'New Deaths (n)'},
            {'label':'Daily Deaths (Smoothed)', 'value':'New Deaths (SMA)'},
        ],
    value='Cases',
    optionHeight = 22
    )
    ],
    xs = 12,
    md = 6, 
)

In [ ]:
##START APP
app = JupyterDash(__name__, external_stylesheets = external_stylesheets)

app.layout = html.Div([
    controls_container,
    tabs
])

@app.callback(
    Output('controls-container', 'children'),
    Input('tabs', 'value')
)
def populate_controls(tab):
    if tab == 'global-tab':
        return controls_global
    return controls_country

@app.callback(
    Output('focus-map', 'figure'),
    [Input('country-dropdown-country', 'value'),
    Input('metric-dropdown-country', 'value')]
)
def update_country_map(iso3, new_metric):
    
    #Helper Function for color scale
    def max_range(x,y):
        return (x if x > y else y)
    
    #Helper Function
    def metric_for_map(new_metric):
        if new_metric == 'New Cases (SMA)' : return 'New Cases (n)'
        if new_metric == 'New Deaths (SMA)' : return 'New Deaths (n)'
        return new_metric
    
    new_metric = metric_for_map(new_metric)
    
    filtered_df = df.loc[(df['Date']  == df['Date'].max())]
    
    #Centering and Zoom height variables
    projection = country_reference.loc[country_reference['ISO-3'] == iso3, 'Projection'].item()
    lat = country_reference.loc[country_reference['ISO-3'] == iso3, 'Lat'].item()
    lon = country_reference.loc[country_reference['ISO-3'] == iso3, 'Lon'].item()
    color_scale_min = filtered_df[new_metric].min()
    color_scale_max = max_range(filtered_df[new_metric].max(), 200)
    
    fig = px.choropleth(filtered_df,
                locations = "ISO-3",               
                color = new_metric,
                hover_name = "Country",  
                color_continuous_scale = 'algae',
                range_color = (color_scale_min, color_scale_max),
                title = '{} by Country'.format(new_metric), 
                template = 'plotly_white'
    )
   
    # Custom fitbounds using center properties
    fig.update_geos(center_lon = lon, center_lat = lat)
    fig.update_geos(lataxis_showgrid=True, lonaxis_showgrid=True)
    
    fig.update_geos(projection_type="natural earth")
    fig.update_layout(height=450, margin={"r":10,"t":30,"l":10,"b":30})
    fig.update_layout(transition_duration=500)
    fig.update_layout(title_x=0.3, title_font_size = 16)
    fig.update_layout(coloraxis_colorbar = dict(thickness = 8))
    fig.update_geos(projection_scale=projection)
    
    return fig

@app.callback(
    [Output('country-card-description', 'children'),
    Output('country-card-list', 'children'),
    Output('region-line', 'figure')],
    [Input('country-dropdown-country', 'value'),
     Input('metric-dropdown-country', 'value')]
)
def update_country_card(iso3, new_metric):
    
    #helper method for card text
    def trend(n): return "increased" if n > 1 else 'decreased'

    new_country =  country_reference.loc[country_reference['ISO-3'] == iso3, 'Country'].iloc[0]
    df_c = df[df['ISO-3'] == iso3]
    last_two_weeks = df_c.loc[:, ['New Deaths (SMA)','New Cases (SMA)', 'Date']].tail(14)
    print(last_two_weeks)
    pct_deaths_two_weeks = round((last_two_weeks.loc[:,'New Deaths (SMA)'].iloc[0] / last_two_weeks.loc[:,'New Deaths (SMA)'].iloc[13] * 100 ) - 100, 2)
    pct_cases_two_weeks = round((last_two_weeks.loc[:,'New Cases (SMA)'].iloc[0] / last_two_weeks.loc[:,'New Cases (SMA)'].iloc[13] * 100) - 100, 2)
    days_since_first_case = (df_c['Date'].max() - df_c['Date'].min()).days
    first_case_date = df_c.loc[(df_c['Cases'] > 0), ['Date']].iloc[0].dt.strftime('%B %d, %Y').iloc[0]
    country_cases = df_c.loc[:,['Cases']].iloc[-1][0]
    country_deaths = df_c.loc[:,['Deaths']].iloc[-1][0]
    country_cases_per_1m = df_c.loc[:,['Deaths per 1M']].iloc[-1][0] 
    country_deaths_per_1m = df_c.loc[:,['Cases per 1M']].iloc[-1][0]

    new_cases = df_c.loc[:,['New Cases (n)']].astype('int').iloc[-1][0]
    under_control = "widespread" if new_cases > 500 else "limited"
    
    
    #Regional Line Chart Variable
    group = country_reference.loc[country_reference['ISO-3'] == iso3, 'Group'].iloc[0]


# Non-COVID19 Country Data
    wfpi_score = country_reference.loc[country_reference['ISO-3'] == iso3, 'WPFI Score'].iloc[0]
    wfpi_rank = country_reference.loc[country_reference['ISO-3'] == iso3, 'WPFI Rank'].iloc[0]
    wfpi_max_rank = country_reference['WPFI Rank'].max()
    country_life_exp = country_reference.loc[country_reference['ISO-3'] == iso3, 'Life Exp 2018'].round(2).iloc[0]

    country_card_text = "{} reported its first case on {}. The country has reported a total of {} cases and {} deaths. \
    Currently, COVID-19 transmission is {} in {}, as the number of new cases per day is {}. \
    In the past two weeks, the average daily case count in {} has {} by {}%, and the death rate has {} by {}%."\
        .format(new_country, first_case_date, country_cases, country_deaths, under_control, new_country, new_cases, new_country,
               trend(pct_cases_two_weeks), pct_cases_two_weeks, trend(pct_deaths_two_weeks), pct_deaths_two_weeks)
    
    country_card_description_content = dbc.Card(
            [html.H3('Country Overview', className = 'card-title'),
            html.P(country_card_text, className = 'card-text')
            ], 
            style = {'border': 'none'}
        )

    country_card_list_content = dbc.ListGroup(
            dbc.ListGroupItem(
                children = [
                    dbc.ListGroupItemHeading(html.H4('{} Summary'.format(new_country))),
                    html.Hr(),
#                     dbc.ListGroupItemHeading(country_card_text),
#                     html.Hr(),
                    dbc.ListGroupItemHeading("Statistics"),
                    dbc.ListGroupItemText(f"Total Cases : {int(country_cases):n}"),
                    dbc.ListGroupItemText(f"Cases per 1M : {int(country_cases_per_1m):n}"),
                    dbc.ListGroupItemText(f"Total Deaths : {int(country_deaths):n}"),
                    dbc.ListGroupItemText(f"Deaths per 1M : {int(country_deaths_per_1m):n}"),
                    dbc.ListGroupItemText(f"Life Expectancy (2018) : {country_life_exp} years"),
                    dbc.ListGroupItemText(f"Press Freedom Index Rank (2020) : {int(wfpi_rank)} out of {int(wfpi_max_rank)}"),
                    dbc.ListGroupItemText(f"United Nations Regional Grouping : {group}")

                ], 
                style = {'color': '#2f4f4f', 'backgroundColor' : '#ede7c6' }
            ),
        flush = True
    )
    
    #Line Chart

    group_isos = country_reference.loc[country_reference['Group'] == group, ['ISO-3']].values.flatten()
    line_chart_df = df.loc[df['ISO-3'].isin(group_isos)]


    line_chart_region = px.line(line_chart_df,
        x = 'Date', 
        y = new_metric, 
        title = '{}, {} Region'.format(new_metric, group), # line_chart_titles[new_metric], 
        color = 'Country',
        text = new_metric, 
        template='gridon', 
        log_y = True,
        labels = dict(
            x = 'Date', 
            y = 'Cases', # str(new_metric).title(),
            country = 'Country')
        )

    line_chart_region.update_traces(mode='lines')
    line_chart_region.update_layout(hovermode='closest', font_size = 14, title_font_size = 16)
    line_chart_region.update_layout(height = 500)
    
    return (country_card_description_content, country_card_list_content, line_chart_region)


import random
a = random.randint(1000,5000)
app.run_server(mode = 'external', debug = True, port = a) 



fitbounds
Code: fig.update_geos(fitbounds=<VALUE>)
Type: enumerated , one of ( False | "locations" | "geojson" )
Determines if this subplot's view settings are auto-computed to fit trace data. On scoped maps, setting `fitbounds` leads to `center.lon` and `center.lat` getting auto-filled. On maps with a non-clipped projection, setting `fitbounds` leads to `center.lon`, `center.lat`, and `projection.rotation.lon` getting auto-filled. On maps with a clipped projection, setting `fitbounds` leads to `center.lon`, `center.lat`, `projection.rotation.lon`, `projection.rotation.lat`, `lonaxis.range` and `lonaxis.range` getting auto-filled. If "locations", only the trace's visible locations are considered in the `fitbounds` computations. If "geojson", the entire trace input `geojson` (if provided) is considered in the `fitbounds` computations, Defaults to "False".

#Dashboard Components
### * Country Level
* Bar chart with new cases (bars) and deaths (red line)
* Choropleth Maps that focuses in on the country??
* Line Charts showing cumulative cases & cumulative deaths in region
* Country profile
  * First Case Date
  * Number of cases
  * Number of deaths
  * last 2 weeks trend
  * Regional grouping - data cleaned and merged
  * Trustwiorthiness data - data cleaned and merged
* Controls
  * Country Selector (single) - Done

### * Global
* Choropleth Map - Done
* Global Fast Facts - In Progress
  * Add 'Country with fastest growing infection rate per 1M'
    * (cases per 1m (today) / cases per 1m (2 weeks ago) * 100
* Mobility Data

<!-- * Top 15 Data Table -->
* Controls
  * Time slider
  * Country Selector (multiple)
  * Metrics Selector

In [ ]:
fig = px.bar(df, x="Date", y="New Deaths (n)", color="Country", title="Long-Form Input")
# fig.show()

In [ ]:
## choropleth 2nd layer (in progress - see choropleth_mapbox (requires api key and geojson) for next steps
    selected_country_df = filtered_df.loc[filtered_df['ISO-3'] == iso3]
    
    fig.add_trace(
        px.choropleth_mapbox(selected_country_df, 
                    locations = "ISO-3",               
                    color = new_metric,
                    hover_name = "Country",  
                    color_continuous_scale = 'algae',
                    range_color = (color_scale_min, color_scale_max),
                    template = 'plotly_white'
                                )
    )

In [ ]:
df = pd.read_csv('/Users/ems/Documents/df.csv', parse_dates = ['Date'])


In [ ]:
#To update the current dataset and pull in fresh data (for joining it to other sheets)

df = pd.read_csv('/Users/ems/Documents/df.csv', parse_dates = ['Date'])

df['Month and Year'] = pd.DatetimeIndex(df['Date']).strftime("%b %Y")

iso3_list = pd.read_csv('Coronavirus_Dashboard/geodata/ISO3.csv')
coords = pd.read_csv('Coronavirus_Dashboard/geodata/country_coordinates.csv').drop(['Country','Alpha-2', 'Num_code'], axis = 1)
un_regions = pd.read_csv('Coronavirus_Dashboard/geodata/UN_Regions.csv', skiprows = 1).drop('Country', axis = 1)
press_freedom_index = pd.read_csv('Coronavirus_Dashboard/geodata/WPFI.csv', skiprows = 1).drop('Country', axis = 1).rename(columns={'Rank': 'WPFI Rank', 'Score': 'WPFI Score'})
life_exp = pd.read_csv('Coronavirus_Dashboard/geodata/life_exp_2018.csv', skiprows = 4).loc[:, ['Country Code', '2018']].rename(columns = {'Country Code': 'ISO-3', '2018': 'Life Exp 2018'})
landmass = pd.read_csv('Coronavirus_Dashboard/geodata/landmass.csv').rename(columns = {'projection':'Projection'}).drop(['Country', 'Rank', 'sq_km', 'zoom_score'], axis = 1)
income_group = pd.read_csv('Coronavirus_Dashboard/geodata/wb_income_group.csv').drop(columns = ['id', 'Lending category', 'Other']).rename(columns = {'Region': 'WB Region', 'Code' : 'ISO-3', 'Income group' : 'Income Group'})
country_reference = iso3_list.merge(un_regions, on = 'ISO-3', how = 'left')\
    .merge(coords, on = 'ISO-3', how = 'left')\
    .merge(press_freedom_index, on = 'ISO-3', how = 'left')\
    .merge(life_exp, how = 'left', on = 'ISO-3')\
    .merge(landmass, on = 'ISO-3', how = 'left')\
    .merge(income_group, on = 'ISO-3', how = 'left')
    
country_reference.to_csv('Coronavirus_Dashboard/geodata/country_reference.csv')


In [ ]:
country_reference

In [ ]:
line_chart_region = dcc.Graph(id = 'line-chart-region')
line_chart_region_df = d

        fig = px.line(line_chart_region_df,
        x = 'Date', 
        y = new_metric, 
        title = line_chart_titles[new_metric], 
        color = 'Country',
        text = new_metric, 
        template='gridon', 
        log_y = False,
        labels = dict(
            x = 'Date', 
            y = str(new_metric).title(),
            country = 'Country')
        )

In [ ]:
import dash_bootstrap_components.themes


In [ ]:
group = country_reference.loc[country_reference['ISO-3'] == 'USA', 'Group'].iloc[0]
countries_in_group = country_reference.loc[country_reference['Group'] == group, ['ISO-3']]


In [ ]:
df['Country']

In [ ]:
group = country_reference.loc[country_reference['ISO-3'] == 'USA', 'Group'].iloc[0]
group_isos = country_reference.loc[country_reference['Group'] == group, ['ISO-3']].values.flatten()
line_chart_df = df.loc[df['ISO-3'].isin(group_isos)]

In [ ]:
df_filtered = df.loc[df['Date'] == df['Date'].max(), ['Cases per 1M', 'Country', 'ISO-3', 'Population']]
income_groups = country_reference.loc[:, ['ISO-3', 'Income Group']]
df_box = df_filtered.merge(income_groups, how = 'left', on = 'ISO-3', copy = False)
df_box['Income Group'].fillna('', inplace = True)

import plotly.graph_objects as go
import plotly.express as px


# fig = px.box(df_box, y="Cases per 1M", points="all", color='Country', verty = False)
# fig.show()
                  
                  
# box = go.Figure(data=)

# box.update_layout(yaxis_type = 'log')
# box.show()

order = ['Low income', 'Lower middle income', 'Upper middle income', 'High income']
fig = px.box(df_box, 
             x="Cases per 1M", 
             points="all", 
             hover_data=['Country'], 
             color = 'Income Group',
#              boxmode = 'overlay',
             orientation = 'h', 
             log_x = True,
             category_orders = {'Income group' : order},
             width = 1000
#              theme = 'plotly white'
            )

fig.show()


In [ ]:
import plotly.graph_objects as go

new_metric = 'Cases per 1M'
fig = go.Figure()
fig.update_layout(margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ))
fig.update_layout(title = str(new_metric) + ' ' + 'Distribution by World Bank Income Group')
fig.update_layout(xaxis_type = 'log', 
#                   height = 800, 
                  legend=dict(
                        x=0,
                        y=1,
                        traceorder='normal',
                        font=dict(size=12),
    ))

colors = ['#3DA5D9', '#73BFB8', '#EA7317', '#D44D5C']
order = ['Low income', 'Lower middle income', 'Upper middle income', 'High income']

i = 0
df_box['pop_norm'] = (40-5) * (df_box['Population'] - df_box['Population'].min()) / (df_box['Population'].max() - df_box['Population'].min()) + 5
print(df_box.describe())
    
for grp in order:

    data = df_box.loc[df_box['Income Group'] == grp]
    data['metric'] = data[new_metric].name
    data['jitter'] = np.random.randint(0,7, size=len(data)) + (20 * i)
    data['pop_norm'].describe()
    #Custom hovertemplate data for scatterplot
    cd = np.stack((data['Country'], data['metric'], data['Income Group']), axis=-1)
    
    
    fig.add_trace(go.Box(
        x = data[new_metric],
        y = data['Income Group'],
        name=str(grp),
#         pointpos=0,
        orientation = 'h',
        boxpoints= None, # represent all points
#         marker_color= "#ffffff",
        line_color=colors[i]
    ))
   
    fig.update_layout( bargap = 1, xaxis_type = 'log')
    fig.update_layout(template='plotly_white')
    
    fig.add_trace(go.Scatter(
        x = data[new_metric],
        y = data['jitter'],
        mode = 'markers', 
        orientation = 'h',
        marker=dict(size = data['pop_norm'], color = colors[i], line = dict( width = 2, color = 'black')),
        showlegend = False, 
        customdata = cd
#         showxaxis = False, 
#         x0=10, 
#         dx=1,
# #         y = [1,2,3,4]
    ))
    fig.update_traces(hovertemplate= "<br>".join(["<b>Country:</b> %{customdata[0]}", 
                                                  "<b>Metric:<b> %{customdata[1]}",
                                                 "<b>Income Group:</b> %{customdata[2]}"]))
         
    i += 1

fig.update_yaxes(visible=False)
fig.show()

# fig.add_trace(go.Box(
#     y=[0.75, 5.25, 5.5, 6, 6.2, 6.6, 6.80, 7.0, 7.2, 7.5, 7.5, 7.75, 8.15,
#         8.15, 8.65, 8.93, 9.2, 9.5, 10, 10.25, 11.5, 12, 16, 20.90, 22.3, 23.25],
#     name="Only Whiskers",
#     boxpoints=False, # no data points
#     marker_color='rgb(9,56,125)',
#     line_color='rgb(9,56,125)',
#     orientation = 'h'
# ))

# fig.add_trace(go.Box(
#     y=[0.75, 5.25, 5.5, 6, 6.2, 6.6, 6.80, 7.0, 7.2, 7.5, 7.5, 7.75, 8.15,
#         8.15, 8.65, 8.93, 9.2, 9.5, 10, 10.25, 11.5, 12, 16, 20.90, 22.3, 23.25],
#     name="Suspected Outliers",
#     boxpoints='suspectedoutliers', # only suspected outliers
#     marker=dict(
#         color='rgb(8,81,156)',
#         outliercolor='rgba(219, 64, 82, 0.6)',
#         line=dict(
#             outliercolor='rgba(219, 64, 82, 0.6)',
#             outlierwidth=2)),
#     line_color='rgb(8,81,156)'
# ))

# fig.add_trace(go.Box(
#     y=[0.75, 5.25, 5.5, 6, 6.2, 6.6, 6.80, 7.0, 7.2, 7.5, 7.5, 7.75, 8.15,
#         8.15, 8.65, 8.93, 9.2, 9.5, 10, 10.25, 11.5, 12, 16, 20.90, 22.3, 23.25],
#     name="Whiskers and Outliers",
#     boxpoints='all', # represent all points
#     marker_color='rgb(107,174,214)',
#     line_color='rgb(107,174,214)'
# ))


In [ ]:
#Horizontal

import plotly.graph_objects as go

new_metric = 'Cases per 1M'

fig = go.Figure()
fig.update_layout(
    margin=dict(
        l=30,
        r=30,
        b=30,
        t=50,
        pad=20
    ),
    yaxis_type = 'log', 
    yaxis_title = 'Cases')
fig.update_layout(title = str(new_metric) + ' ' + 'Distribution by World Bank Income Group')
fig.update_layout( legend=dict(
        x=0,
        y=1,
        traceorder='normal',
        font=dict(
            size=12,),
    ))

colors = ['#3DA5D9', '#73BFB8', '#EA7317', '#D44D5C']
order = ['Low income', 'Lower middle income', 'Upper middle income', 'High income']

i = 0
df_box['pop_norm'] = (40-5) * (df_box['Population'] - df_box['Population'].min()) / (df_box['Population'].max() - df_box['Population'].min()) + 5
print(df_box.describe())
    
for grp in order:

    data = df_box.loc[df_box['Income Group'] == grp]
    data['metric'] = data[new_metric].name
    data['jitter'] = np.random.randint(0,7, size=len(data)) + (20 * i)
    data['jitter'] = np.where(data['Population'] == data['Population'].mean(), data['jitter'].median(), data['jitter'] )
    data['pop_norm'].describe()
    #Custom hovertemplate data for scatterplot
    cd = np.stack((data['Country'], data['metric'], data['Income Group'], data['metric']), axis=-1)
    
    
    fig.add_trace(go.Box(
        x = data['Income Group'],
        y = data[new_metric],
        name=str(grp),
        orientation = 'v',
        pointpos= 0, # Shows data point 
        boxpoints= None, # represent all points
        fillcolor = colors[i],
        line=dict( width = 2, color = 'black'),
        width = 1

    ))
   
    fig.update_layout( bargap = 1)

    fig.update_layout(template='plotly_white')
    
    fig.add_trace(go.Scatter(
        y = data[new_metric],
        x = data['jitter'],
        mode = 'markers', 
        orientation = 'v',
        marker=dict(size = data['pop_norm'], color = colors[i], line = dict( width = 2, color = 'black')),
        showlegend = False, 
        customdata = cd
#         showxaxis = False, 
#         x0=10, 
#         dx=1,
# #         y = [1,2,3,4]
    ))
    fig.update_traces(hovertemplate= "<br>".join(["<b>Country</b>: %{customdata[0]}", 
                                                  "<b>customdata[3]}</b>: %{customdata[1]}",
                                                 "<b>Income Group</b>: %{customdata[2]}"]))
         
    i += 1

fig.update_xaxes(visible=False)

fig.show()

# fig.add_trace(go.Box(
#     y=[0.75, 5.25, 5.5, 6, 6.2, 6.6, 6.80, 7.0, 7.2, 7.5, 7.5, 7.75, 8.15,
#         8.15, 8.65, 8.93, 9.2, 9.5, 10, 10.25, 11.5, 12, 16, 20.90, 22.3, 23.25],
#     name="Only Whiskers",
#     boxpoints=False, # no data points
#     marker_color='rgb(9,56,125)',
#     line_color='rgb(9,56,125)',
#     orientation = 'h'
# ))

# fig.add_trace(go.Box(
#     y=[0.75, 5.25, 5.5, 6, 6.2, 6.6, 6.80, 7.0, 7.2, 7.5, 7.5, 7.75, 8.15,
#         8.15, 8.65, 8.93, 9.2, 9.5, 10, 10.25, 11.5, 12, 16, 20.90, 22.3, 23.25],
#     name="Suspected Outliers",
#     boxpoints='suspectedoutliers', # only suspected outliers
#     marker=dict(
#         color='rgb(8,81,156)',
#         outliercolor='rgba(219, 64, 82, 0.6)',
#         line=dict(
#             outliercolor='rgba(219, 64, 82, 0.6)',
#             outlierwidth=2)),
#     line_color='rgb(8,81,156)'
# ))

# fig.add_trace(go.Box(
#     y=[0.75, 5.25, 5.5, 6, 6.2, 6.6, 6.80, 7.0, 7.2, 7.5, 7.5, 7.75, 8.15,
#         8.15, 8.65, 8.93, 9.2, 9.5, 10, 10.25, 11.5, 12, 16, 20.90, 22.3, 23.25],
#     name="Whiskers and Outliers",
#     boxpoints='all', # represent all points
#     marker_color='rgb(107,174,214)',
#     line_color='rgb(107,174,214)'
# ))


In [39]:
# Testing (again) db api methods
#!/usr/bin/env python
# coding: utf-8

# In[23]:


import psycopg2
from io import StringIO
import os
import pandas as pd
import requests
import os

from process_data import json_to_df, create_chart_ready_df


def connect():
    DATABASE_URL = os.environ['DATABASE_URL']
    
    # Connect to the PostgreSQL database server 
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(DATABASE_URL, sslmode='require')
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn


def get_from_db(table):

    try:
        conn = connect()
        Cursor = conn.cursor()
        
    except Exception as e:
        print('Unable to connect to db. Please check credentials.')
        return e
        
    try:
        Cursor.execute('Select * from {}'.format(table))
    except Exception as e:
        print('Unable to execute SQL query. Rolling back query...')
        Cursor.execute('rollback')
        return e
        
    results = Cursor.fetchall()
    Cursor.close()
    return results

    
def set_to_db (df, table):
    conn = connect()
    Cursor = conn.cursor()
    try:
        Cursor.execute('DELETE FROM {};'.format(table))
        print('Successfully cleared cached data')
    except Exception as e:
        print(f'Failed to clear cached data : {e} ')
    copy_from_stringio(conn, df, table)
    Cursor.close()
    
    
#helper method to set data to db
def copy_from_stringio(conn, df, table):
    """
    Here we are going save the dataframe in memory 
    and use copy_from() to copy it to the table
    """
    # save dataframe to an in memory buffer
    buffer = StringIO()
    df.to_csv(buffer, index_label = 'id', header=False)
    print(df.columns)
    buffer.seek(0)
    
    cursor = conn.cursor()
    try:
        cursor.copy_from(buffer, table, sep=",")
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("Successfully posted to DB")
    cursor.close()
    
#API get method
def get_from_api():
    try:
        json_url = 'https://coronavirus-tracker-api.herokuapp.com/v2/locations?timelines=1'
        response = requests.get(json_url)

        print(response)
        timeline_json = response.json()
        return timeline_json
    
    except Exception as e:
        print('Error making API call: ', e)
        return e

In [24]:
url = 'https://jsonplaceholder.typicode.com/todos/1'
url2 = 'https://coronavirus-tracker-api.herokuapp.com/v2/locations'

In [41]:
# data = get_from_api()
# new_df = json_to_df(data)
new_df

,Date,Deaths,Cases,Country,Country Code,Population,Province,Latitude,Longitude,Cases per 1M,Deaths per 1M,New Deaths (n),New Cases (n),Multiple_Territories,Month and Year,Day,ISO-3
0,2020-01-22,0,0,Afghanistan,AF,37172386,,33.93911,67.709953,0.0,0.0,NaN,NaN,False,Jan 2020,22,AFG
1,2020-01-23,0,0,Afghanistan,AF,37172386,,33.93911,67.709953,0.0,0.0,0.0,0.0,False,Jan 2020,23,AFG
2,2020-01-24,0,0,Afghanistan,AF,37172386,,33.93911,67.709953,0.0,0.0,0.0,0.0,False,Jan 2020,24,AFG
3,2020-01-25,0,0,Afghanistan,AF,37172386,,33.93911,67.709953,0.0,0.0,0.0,0.0,False,Jan 2020,25,AFG
4,2020-01-26,0,0,Afghanistan,AF,37172386,,33.93911,67.709953,0.0,0.0,0.0,0.0,False,Jan 2020,26,AFG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97642,2021-01-14,636,25368,Zimbabwe,ZW,14439018,,-19.015438,29.154857,1756.9,44.0,47.0,1112.0,False,Jan 2021,14,ZWE
97643,2021-01-15,666,26109,Zimbabwe,ZW,14439018,,-19.015438,29.154857,1808.2,46.1,30.0,741.0,False,Jan 2021,15,ZWE
97644,2021-01-16,683,26881,Zimbabwe,ZW,14439018,,-19.015438,29.154857,1861.7,47.3,17.0,772.0,False,Jan 2021,16,ZWE
97645,2021-01-17,713,27203,Zimbabwe,ZW,14439018,,-19.015438,29.154857,1884.0,49.4,30.0,322.0,False,Jan 2021,17,ZWE


In [37]:
# data['locations']['id' == 0]

{'id': 0,
 'country': 'Afghanistan',
 'country_code': 'AF',
 'country_population': 37172386,
 'province': '',
 'last_updated': '2021-01-19T12:40:46.737392Z',
 'coordinates': {'latitude': '33.93911', 'longitude': '67.709953'},
 'latest': {'confirmed': 54062, 'deaths': 2343, 'recovered': 46359}}

In [38]:
for row in data['locations']:
    print(row)

{'id': 0, 'country': 'Afghanistan', 'country_code': 'AF', 'country_population': 37172386, 'province': '', 'last_updated': '2021-01-19T12:40:46.737392Z', 'coordinates': {'latitude': '33.93911', 'longitude': '67.709953'}, 'latest': {'confirmed': 54062, 'deaths': 2343, 'recovered': 46359}}
{'id': 1, 'country': 'Albania', 'country_code': 'AL', 'country_population': 2866376, 'province': '', 'last_updated': '2021-01-19T12:40:46.883130Z', 'coordinates': {'latitude': '41.1533', 'longitude': '20.1683'}, 'latest': {'confirmed': 67982, 'deaths': 1281, 'recovered': 40870}}
{'id': 2, 'country': 'Algeria', 'country_code': 'DZ', 'country_population': 42228429, 'province': '', 'last_updated': '2021-01-19T12:40:47.001421Z', 'coordinates': {'latitude': '28.0339', 'longitude': '1.6596'}, 'latest': {'confirmed': 104092, 'deaths': 2840, 'recovered': 70747}}
{'id': 3, 'country': 'Andorra', 'country_code': 'AD', 'country_population': 77006, 'province': '', 'last_updated': '2021-01-19T12:40:47.072436Z', 'coor